# Pose Estimation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensein/senselab/blob/main/tutorials/video/pose_estimation.ipynb)

This tutorial demonstrates how to use Senselab's Pose Estimation API for estimating human poses in images. Senselab supports multiple pose estimation backends, such as MediaPipe and YOLO.

## Setup

Let's get started by installing Senselab and importing the necessary modules from Senselab for processing images and performing pose estimation.

In [ ]:
%pip install senselab

In [ ]:
from senselab.video.tasks.pose_estimation import MediaPipePoseEstimator, YOLOPoseEstimator
from senselab.video.data_structures.pose import ImagePose

In [ ]:
!mkdir -p tutorial_images
!wget -O tutorial_images/no_people.jpg https://github.com/sensein/senselab/raw/main/src/tests/data_for_testing/pose_data/no_people.jpg
!wget -O tutorial_images/single_person.jpg https://github.com/sensein/senselab/raw/main/src/tests/data_for_testing/pose_data/single_person.jpg
!wget -O tutorial_images/three_people.jpg https://github.com/sensein/senselab/raw/main/src/tests/data_for_testing/pose_data/three_people.jpg

## MediaPipe Pose Estimation

### Initialize MediaPipe Estimator
Let's initialize the MediaPipe Pose Estimator. We will use the "full" model for this tutorial.


In [ ]:
mp_estimator = MediaPipePoseEstimator(model_type="full")

### Perform Pose Estimation
Now, let's perform pose estimation on the example image.

In [ ]:
image_path = "tutorial_images/single_person.jpg"
result = mp_estimator.estimate_from_path(image_path)

# Check the output
print(f"Number of individuals detected: {len(result.individuals)}")

### Visualize Results
To visualize the estimated poses, use Senselab's built-in plotting utilities.

In [ ]:
#TODO

## YOLO Pose Estimation

### Initialize YOLO Estimator
Next, let's initialize the YOLO Pose Estimator using the "11n" model.

In [ ]:
yolo_estimator = YOLOPoseEstimator(model="8n")

### Perform Pose Estimation
Run the YOLO model on the same example image.

In [ ]:
result = yolo_estimator.estimate_from_path(image_path)

# Check the output
print(f"Number of individuals detected: {len(result.individuals)}")

### Visualize Results
Plot the YOLO-estimated poses on the image.

In [ ]:
plot_image_pose(result)

## Extended Cases

### Estimating Poses in Multiple-Person Images

You can specify the maximum number of individuals to detect using the num_individuals parameter (MediaPipe only):

In [ ]:
multi_person_image = "tutorial_images/three_people.jpg"
result = mp_estimator.estimate_from_path(multi_person_image, num_individuals=3)
plot_image_pose(result)

### Handling No Person Detected
If no person is detected in the image, the output will have zero individuals.

In [ ]:
no_person_image = "tutorial_images/no_people.jpeg"
result = mp_estimator.estimate_from_path(no_person_image)

if len(result.individuals) == 0:
    print("No individuals detected in the image.")